# What is MNIST dataset?
Image result for mnist dataset
The MNIST dataset is an acronym that stands for the Modified National Institute of Standards and Technology dataset. It is a dataset of 60,000 small square 28×28 pixel grayscale images of handwritten single digits between 0 and 9

In [1]:
'''
[] config
[] hyper parameter 
[] dataset
[] dataloader
[] data visualization
[] model designing
[] criterion and optimizer
[] training loop (with batch)
'''

'\n[] config\n[] hyper parameter \n[] dataset\n[] dataloader\n[] data visualization\n[] model designing\n[] criterion and optimizer\n[] training loop (with batch)\n'

In [2]:
# imports

import torch
import torch.nn as nn

from torchvision  import datasets,transforms
import torchvision
from torch.utils.data import dataset,dataloader

import matplotlib.pyplot as plot




# tensor board

In [3]:
############## TENSORBOARD ########################
import sys
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/mnist1')
###################################################

In [4]:
# configs

# device config

device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device=device_name)
print(f"device used: {device}")

device used: cpu


In [5]:
# hyper parameters

input_layer_nodes = 28*28 # input grid size

hidden_layer_nodes = 100 # used for improving classification

output_layer_nodes = 10 # total number of classes

batch_size = 64 # fast training use 32 or 64

learning_rate = 0.01 # learning rate

number_of_epochs = 2 # number of epochs





# Data from Dataset

In [6]:
class Reshape():

    def __init__(self,row,col) -> None:
        self.row = row
        self.col = col
    
    def __call__(self,single_sample):
        # print(single_sample)
        features = single_sample

        features  = features.view(-1,self.row*self.col)

        return features


composed_transform = transforms.Compose(transforms=[
    transforms.ToTensor(),
    Reshape(28,28)
])


In [7]:
train_dataset = datasets.MNIST(
    root="./data/mnist/",
    download=True,
    train=True,
    transform= composed_transform
    )
# train_dataset.data.shape
# train_dataset.__getitem__(0)

In [8]:
test_dataset = datasets.MNIST(
    root='./data/mnist/',
    train=False,
    download=True,
    transform=transforms.transforms.ToTensor()
)
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [9]:
train_loader = dataloader.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
    )

test_loader = dataloader.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    
)

In [10]:
all_sample = iter(train_loader)

first_sample = all_sample.next()

sample,label = first_sample

print(f"sample size : {sample.shape}")
print(f"label size: {label.shape}")

# print(f"sample : {sample}")
# print(f"label: {label}")



sample size : torch.Size([64, 1, 784])
label size: torch.Size([64])


In [11]:
sample.shape
example_sample = sample.view(-1,1,28,28)
example_sample.shape

torch.Size([64, 1, 28, 28])

In [12]:
for i in range(6):
    plot.subplot(2,3,i+1)
    plot.imshow(sample[i][0].view(28,28),cmap="gray")
plot.show()

In [13]:

############## TENSORBOARD ########################
img_grid = torchvision.utils.make_grid(example_sample)
writer.add_image('mnist_images', img_grid)
writer.close()
###################################################

# Designing Neural Network

In [14]:
# Neural Network

class NeuralNetork(nn.Module):

    def __init__(self,input_layer_nodes,hidden_layer_nodes,output_layer_nodes) -> None:
        super(NeuralNetork,self).__init__()

    
        self.input_layer = nn.Linear(in_features=input_layer_nodes,out_features=hidden_layer_nodes)
        
        self.relu_activation = nn.ReLU()

        self.hiden_layer = nn.Linear(in_features=hidden_layer_nodes,out_features=output_layer_nodes)

        # no softmax

    
    def forward(self,batch_samples):
        out = batch_samples.view(-1,28*28)
        # print(out.shape)
        out = self.input_layer(out)

        out = self.relu_activation(out)

        out = self.hiden_layer(out)

        return out

model = NeuralNetork(input_layer_nodes=input_layer_nodes,hidden_layer_nodes=hidden_layer_nodes,output_layer_nodes=output_layer_nodes)

In [15]:
############## TENSORBOARD ########################
writer.add_graph(model, example_sample.reshape(-1, 28*28))
writer.close()
#sys.exit()
###################################################


# criterian and optimizer

In [16]:

criterian = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

# Training Loop

In [17]:
# Train the model
running_loss = 0.0
running_correct = 0
n_total_steps = len(train_loader)
for epoch in range(number_of_epochs):

    for i,(sample,labels) in enumerate(train_loader):
        # making grad zero for each next batch size
        optimizer.zero_grad()

        # sample = sample.view(-1,28*28).to(device)
        # print(sample.shape)
        sample = sample.to(device)
        # print(sample.shape)

        labels = labels.to(device)

        y_predict = model(sample)

        loss = criterian(y_predict,labels)

        loss.backward()

        optimizer.step()

        running_loss = loss
        
        _, predicted = torch.max(y_predict.data, 1)
        running_correct += (predicted == labels).sum().item()

        if (i+1) % 100 == 0:
            
            print(f"-------------------- iteration --{i+1} ----------------")

            print(f"loss: {loss:.03f}")
                ############## TENSORBOARD ########################
            writer.add_scalar('training loss', running_loss / 100, epoch * n_total_steps + i)
            running_accuracy = running_correct / (100 * predicted.size(0))
            writer.add_scalar('accuracy', running_accuracy, epoch * n_total_steps*predicted.size(0) + i)
            running_correct = 0
            running_loss = 0.0
            ###################################################

-------------------- iteration --100 ----------------
loss: 0.088
-------------------- iteration --200 ----------------
loss: 0.260
-------------------- iteration --300 ----------------
loss: 0.393
-------------------- iteration --400 ----------------
loss: 0.275
-------------------- iteration --500 ----------------
loss: 0.134
-------------------- iteration --600 ----------------
loss: 0.133
-------------------- iteration --700 ----------------
loss: 0.097
-------------------- iteration --800 ----------------
loss: 0.121
-------------------- iteration --900 ----------------
loss: 0.105
-------------------- iteration --100 ----------------
loss: 0.060
-------------------- iteration --200 ----------------
loss: 0.034
-------------------- iteration --300 ----------------
loss: 0.095
-------------------- iteration --400 ----------------
loss: 0.209
-------------------- iteration --500 ----------------
loss: 0.097
-------------------- iteration --600 ----------------
loss: 0.141
----------

# Testing

In [18]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
class_labels = []
class_preds = []
n_samples = 0
n_correct = 0
with torch.no_grad():
    for (batch_samples,label) in iter(test_loader):

        batch_samples = batch_samples.to(device)
        y_predict = model(batch_samples)

        _,prediction = torch.max(y_predict,dim=1)

        n_correct += (prediction == label).sum()

        n_samples += batch_samples.shape[0]


        class_probs_batch = [F.softmax(output, dim=0) for output in y_predict]

        class_preds.append(class_probs_batch)
        class_labels.append(prediction)


    


    
    # 10000, 10, and 10000, 1
    # stack concatenates tensors along a new dimension
    # cat concatenates tensors in the given dimension
    class_preds = torch.cat([torch.stack(batch) for batch in class_preds])
    class_labels = torch.cat(class_labels)

    ############## TENSORBOARD ########################
    classes = range(10)
    for i in classes:
        labels_i = class_labels == i
        preds_i = class_preds[:, i]
        writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
        writer.close()
    ###################################################
    acc = n_correct/ n_samples
    print(f"accuracy: {acc}")

    

accuracy: 0.9621000289916992
